In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, Model
import matplotlib.pyplot as plt

# 데이터 로드
data = pd.read_csv('C:/Users/3333c/Desktop/학교/3-2/딥러닝응용/creditcard.csv')
data

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


In [4]:
# 정상 데이터와 비정상 데이터 분리
normal_data = data[data['Class'] == 0]  # 정상 거래
fraud_data = data[data['Class'] == 1]   # 사기 거래

# 'Class' 열을 제외한 특성들만 선택
features = data.drop('Class', axis=1)

In [6]:
# 데이터 스케일링
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# 정상 데이터와 비정상 데이터 분리 (스케일링된 데이터)
normal_features = features_scaled[data['Class'] == 0]
fraud_features = features_scaled[data['Class'] == 1]

# 정상 데이터만으로 학습/테스트 분할
X_train, X_test_normal = train_test_split(normal_features, test_size=0.2, random_state=42)

In [8]:
# 테스트 데이터에 비정상 데이터 추가
X_test = np.concatenate([X_test_normal, fraud_features])
y_test = np.concatenate([np.zeros(len(X_test_normal)), np.ones(len(fraud_features))])


In [10]:
# 오토인코더 모델 구축
input_dim = X_train.shape[1]

input_layer = layers.Input(shape=(input_dim,))
encoded = layers.Dense(64, activation='relu')(input_layer)
encoded = layers.Dense(32, activation='relu')(encoded)
encoded = layers.Dense(16, activation='relu')(encoded)

decoded = layers.Dense(32, activation='relu')(encoded)
decoded = layers.Dense(64, activation='relu')(decoded)
decoded = layers.Dense(input_dim, activation='linear')(decoded)

autoencoder = Model(input_layer, decoded)
autoencoder.compile(optimizer='adam', loss='mse')

In [ ]:
# 모델 학습
history = autoencoder.fit(
    X_train, X_train,
    epochs=50,
    batch_size=32,
    validation_split=0.1,
    verbose=1
)

Epoch 1/50
6398/6398 ━━━━━━━━━━━━━━━━━━━━ 6s 675us/step - loss: 0.3844 - val_loss: 0.1213
Epoch 2/50
6398/6398 ━━━━━━━━━━━━━━━━━━━━ 4s 648us/step - loss: 0.1114 - val_loss: 0.0912
Epoch 3/50
6398/6398 ━━━━━━━━━━━━━━━━━━━━ 4s 646us/step - loss: 0.0834 - val_loss: 0.0819
Epoch 4/50
6398/6398 ━━━━━━━━━━━━━━━━━━━━ 4s 653us/step - loss: 0.0713 - val_loss: 0.0617
Epoch 5/50
6398/6398 ━━━━━━━━━━━━━━━━━━━━ 4s 622us/step - loss: 0.0660 - val_loss: 0.0619
Epoch 6/50
6398/6398 ━━━━━━━━━━━━━━━━━━━━ 4s 618us/step - loss: 0.0617 - val_loss: 0.0590
Epoch 7/50
6398/6398 ━━━━━━━━━━━━━━━━━━━━ 4s 616us/step - loss: 0.0587 - val_loss: 0.0921
Epoch 8/50
6398/6398 ━━━━━━━━━━━━━━━━━━━━ 4s 627us/step - loss: 0.0532 - val_loss: 0.0703
Epoch 9/50
6398/6398 ━━━━━━━━━━━━━━━━━━━━ 4s 619us/step - loss: 0.0607 - val_loss: 0.0483
Epoch 10/50
6398/6398 ━━━━━━━━━━━━━━━━━━━━ 4s 612us/step - loss: 0.0506 - val_loss: 0.0520
Epoch 11/50
6398/6398 ━━━━━━━━━━━━━━━━━━━━ 4s 622us/step - loss: 0.0520 - val_loss: 0.0429
Epoch 12

In [ ]:
# 재구성 오차 계산
reconstruction_errors = np.mean(np.power(X_test - autoencoder.predict(X_test), 2), axis=1)

In [ ]:
# threshold 설정 (예: 95퍼센타일)
threshold = np.percentile(
    np.mean(np.power(X_train - autoencoder.predict(X_train), 2), axis=1),
    95
)